# Mission 3: Orchestrate SQL + AI workflows

This mission advances your skills by teaching you how to orchestrate complex workflows that combine SQL database operations with AI capabilities. You'll learn to build intelligent, multi-step processes that seamlessly integrate data retrieval, transformation, and AI-powered decision making.

## Overview
Learn how to design and implement sophisticated AI workflows that coordinate multiple database queries, embedding operations, and language model calls to solve complex business problems requiring multiple steps and decision points.


## Prerequisites
1. Mission 1 completed (embeddings and vector search)
1. Mission 2 completed (RAG implementation)
1. Embedding and language model access

In [1]:
import mssql_python
import dotenv

# Load environment variables from .env file
dotenv.load_dotenv("../.env")

# Establish a connection
# Specify connection string (semicolon-delimited key=value format preserved)
connection_string = dotenv.get_key("../.env", "SERVER_CONNECTION_STRING")
connection = mssql_python.connect(connection_string)

# Execute a realistic query against AdventureWorksLT:
# Top 10 customers by number of orders, with their total spend
cursor = connection.cursor()
cursor.execute("""
    SELECT TOP 10 *
    FROM dbo.walmart_ecommerce_product_details AS w

""")
rows = cursor.fetchall()

for row in rows:
    print(row)

# Close the connection
cursor.close()


(1, '00068601cf9047122e9a9f8b300a763b', '2019-03-22 17:56:41 +0000', 'https://www.walmart.com/ip/Ebe-Men-Black-Rectangle-Half-Rim-Spring-Hinge-Eyewear-Reading-Glasses-2036/199534909', 'Ebe Men Black Rectangle Half Rim Spring Hinge Eyewear Reading Glasses 2036', 'Count on EBE for all of your eye correction needs this light weight sleek looking pair of glasses are something to be won. We guarantee you will love the style of these frames with the sharply colored temples on the side of your face give a great fashion look and a great comfort your face we guarantee your satisfaction.| Always custom made to your individual order specifications in our FDA Listed Lab with the remarkable Optical grade Custom frames and Lenses available. Better style better grade and always less than retail. Rx lens included! Glasses come with a Hard protective case and micro fiber cleaning cloth. Very light and strong frame with Ebe signature comfort fit bridge for easy all day use without hot spots. We design a

In [4]:
from langchain.agents import create_agent
from langchain_azure_ai.chat_models import AzureAIChatCompletionsModel
from langchain_azure_ai.embeddings import AzureAIEmbeddingsModel


chat_model = AzureAIChatCompletionsModel(
    endpoint=dotenv.get_key("../.env", "MODEL_ENDPOINT_URL"),
    credential=dotenv.get_key("../.env", "MODEL_API_KEY"),
    model="gpt-4.1",)

response = chat_model.invoke("Say hello")
print(response.content)

embed_model = AzureAIEmbeddingsModel(
    model="text-embedding-ada-002",
    endpoint=dotenv.get_key("../.env", "MODEL_ENDPOINT_URL"),
    credential=dotenv.get_key("../.env", "MODEL_API_KEY"),
)



def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

agent = create_agent(
    model=chat_model,
    tools=[get_weather],
    system_prompt="You are a helpful assistant",
)

# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
)

C:\Users\jasmineg\AppData\Local\Temp\ipykernel_34724\1925643417.py:6: ExperimentalWarning: AzureAIChatCompletionsModel is currently in preview and is subject to change. This preview is provided without a service-level agreement, and we don't recommend it for production workloads. Certain features might not be supported or might have constrained capabilities. For more information, see https://azure.microsoft.com/support/legal/preview-supplemental-terms
  chat_model = AzureAIChatCompletionsModel(


Hello! How can I help you today?


C:\Users\jasmineg\AppData\Local\Temp\ipykernel_34724\1925643417.py:14: ExperimentalWarning: AzureAIEmbeddingsModel is currently in preview and is subject to change. This preview is provided without a service-level agreement, and we don't recommend it for production workloads. Certain features might not be supported or might have constrained capabilities. For more information, see https://azure.microsoft.com/support/legal/preview-supplemental-terms
  embed_model = AzureAIEmbeddingsModel(


{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='1bdee603-4c48-4e80-bd92-87929c6cd8f8'),
  AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'gpt-4.1-2025-04-14', 'token_usage': {'input_tokens': 56, 'output_tokens': 16, 'total_tokens': 72}, 'finish_reason': 'tool_calls'}, id='lc_run--019c1f64-58e1-71f3-a14e-f6e52b933876-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'San Francisco'}, 'id': 'call_8QoaOMEqGbLK87KZPVFQ41F3', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 56, 'output_tokens': 16, 'total_tokens': 72}),
  ToolMessage(content="It's always sunny in San Francisco!", name='get_weather', id='0bb048a0-9e6c-44bf-a311-d3aa58cd6811', tool_call_id='call_8QoaOMEqGbLK87KZPVFQ41F3'),
  AIMessage(content='The weather in San Francisco is described as "always sunny." If you need a more detailed or current forecast, let me know!', additional_kwargs={}, response_metada

In [5]:
def find_products(search_term: str) -> str:
    """Search the product catalog for items matching keywords. Use this to find products when users ask about buying or looking for items."""
    cursor = connection.cursor()
    sql = """SET NOCOUNT ON;
    DECLARE @out nvarchar(max);
    EXEC [dbo].[get_similar_items] @inputText = ?, @error = @out OUTPUT;
    SELECT @out AS error;"""

    params = (search_term)
    cursor.execute(sql, params)
    results = []
    while True:
        rows = cursor.fetchall()
        results.extend(rows)
        if not cursor.nextset():
            break
    cursor.close()
    return str(results)  # Return the data

print(find_products("laptop"))

product_agent = create_agent(
    model=chat_model,
    tools=[find_products],
    system_prompt="""You are a product assistant who helps users find the best products available in the catalog.
                IMPORTANT: Work with whatever information the user provides and make reasonable assumptions when needed.
                Interpret the user's request directly, then use the find_products tool to get relevant products.               
                Use only the information available in the provided JSON to answer the question.
                Don't return products that are not relevant with the ask or that don't comply with user request.
                Make sure to use details, notes, and description that are provided in each product are used only with that product.                
                If the question cannot be answered by the provided samples, don't return any result.
                If asked question is about topics you don't know, don't return any result.
                If no products are provided, don't return any result.""",
)


product_agent.invoke(
    {"messages": [{"role": "user", "content": "I'm looking for a gaming laptop with high performance."}]}
)

[('[{"id":6532,"name":"Lenovo THINKPAD T440P 20AN0072IX Laptop Screen 14 SLIM LED BOTTOM RIGHT WXGA HD","description":"Lenovo THINKPAD T440P 20AN0072IX Laptop Screen 14 SLIM LED BOTTOM RIGHT WXGA HD|Please match your original screen\'s BACKLIGHT TYPE, SIZE, and RESOLUTION before you purchase this item, because some laptop models come with more than one type of screens."},{"id":10159,"name":"Refurbished Dell Latitude E6420 Laptop i5 Dual-Core 8GB 1TB Win 7 Pro B v.BB","description":"Dell Latitude E6420 Laptop 2.50GHz Minimum i5 Dual-Core Processor Speed 8GB 1TB Win 7 Pro Professionally Reconditioned with Warranty - Moderate Signs of Use Cosmetic Condition: Fair Condition - These laptops have moderate cosmetic scratches or scuffs on the bezel and casing and show signs of use on keyboard, trackpad and other highly utilized parts. The expansion\\/SD slot covers may not be present. The screen may have no more than 2 of the following: light scratches, 1-2 dead pixels, or bruises no larger th

{'messages': [HumanMessage(content="I'm looking for a gaming laptop with high performance.", additional_kwargs={}, response_metadata={}, id='4a6a22ca-11e7-48c8-9773-0b7c582a57b0'),
  AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'gpt-4.1-2025-04-14', 'token_usage': {'input_tokens': 229, 'output_tokens': 19, 'total_tokens': 248}, 'finish_reason': 'tool_calls'}, id='lc_run--019c1f64-6eca-7251-b4f3-369e61f3a0c3-0', tool_calls=[{'name': 'find_products', 'args': {'search_term': 'high performance gaming laptop'}, 'id': 'call_hk1J1t1ZwkoFSG4EnZKqisR6', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 229, 'output_tokens': 19, 'total_tokens': 248}),
  ToolMessage(content='[(\'[{"id":18691,"name":"Refurbished DELL PRECISION M5510 I7 6820HQ 3.6GHZ QUADRO M1000M 2GB 16GB Ram 2133MHZ 4K 3840X2160 TOUCH 512GB SSD Windows 10","description":"|Refurbished DELL PRECISION M5510 I7 6820HQ 3.6GHZ QUADRO M1000M 2GB 16GB Ram 2133MHZ 4K 3840X2160 TOUCH 5